In [1]:
USER_FLAG= "--user"
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.7.0 --upgrade
!pip3 install {USER_FLAG} kfp==1.8.9 google-cloud-pipeline-components==0.2.0

In [2]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google.client import AIPlatformClient

In [3]:
PROJECT_ID = 'digital-proton-368805'
REGION = 'us-central1'
BUCKET_NAME = 'gs://vertext-ai-piepline'
PIPELINE_ROOT = "{}/pipeline_root2/".format(BUCKET_NAME)

In [4]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [5]:
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcp_aip

# steps
* dataset
* training
* endpoint
* deployment

In [16]:
@dsl.pipeline(name="mlpipeline", pipeline_root = PIPELINE_ROOT)
def mlpipeline():
    
    image_data = gcp_aip.ImageDatasetCreateOp(display_name='Shoe_vs_Sandal_vs_Boot_Dataset_2',
                                              gcs_source='gs://automl_image_dataset/Shoe_vs_Sandal_vs_Boot_Dataset-1/image-classify-dataset.csv',
                                              import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
                                              project = PROJECT_ID
                                             )
    
    
    training_out = gcp_aip.AutoMLImageTrainingJobRunOp(project = PROJECT_ID,
                                        display_name='Shoe_vs_Sandal_vs_Boot_Training_2',
                                        prediction_type='classification',
                                        dataset = image_data.outputs['dataset'],
                                        budget_milli_node_hours=8000)
    
    
    
    endpoint_out = gcp_aip.EndpointCreateOp(project = PROJECT_ID,
                             location=REGION,
                             display_name='image_endpoint'
                            )
    
    gcp_aip.ModelDeployOp(model= training_out.outputs["model"],
                          endpoint = endpoint_out.outputs["endpoint"],
                          automatic_resources_min_replica_count=1,
                          automatic_resources_max_replica_count=1)

In [17]:
compiler.Compiler().compile(pipeline_func=mlpipeline, package_path="p2.json")

In [18]:
job = aiplatform.PipelineJob(
    display_name="my-ml-pipeline",
    template_path="p2.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=True,
    project=PROJECT_ID
)

In [ ]:
job.submit()